In [ ]:
!pip install transformers accelerate peft datasets trl psutil GPUtil

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 27.8 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.5/561.5 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 21.7 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.26.5
    Uninstalling huggingface-hub-0.26.5:
      Successfully uninstalled huggingface-hub-0.26.5
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.2.0
    Uninstalling accelerate-1.2.0:
      Successfully uninstalled accelerate-1.2.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.47.0
    Uninstalling transformers-4.47.0:
      Successfully uninstalled transformers-4.47.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
docling-ibm-models 3.4.2 requires numpy<3.

In [7]:
# Import necessary libraries
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model
from datasets import load_dataset
from trl import SFTTrainer
import psutil
import GPUtil
import time
import threading
from datetime import datetime

# 메모리 모니터링 함수
def monitor_memory(stop_event):
    """실시간으로 메모리 사용량을 모니터링하는 함수"""
    while not stop_event.is_set():
        try:
            # 시스템 메모리
            memory = psutil.virtual_memory()
            print(f"\n[{datetime.now().strftime('%H:%M:%S')}] 시스템 메모리: "
                  f"사용: {memory.used / 1024**3:.1f}GB / "
                  f"전체: {memory.total / 1024**3:.1f}GB "
                  f"({memory.percent:.1f}%)")
            
            # GPU 메모리 (사용 가능한 경우)
            try:
                gpus = GPUtil.getGPUs()
                for i, gpu in enumerate(gpus):
                    print(f"[{datetime.now().strftime('%H:%M:%S')}] GPU {i}: "
                          f"메모리 사용: {gpu.memoryUsed}MB / "
                          f"전체: {gpu.memoryTotal}MB "
                          f"({gpu.memoryUtil*100:.1f}%)")
            except:
                # PyTorch로 GPU 메모리 확인
                if torch.cuda.is_available():
                    for i in range(torch.cuda.device_count()):
                        allocated = torch.cuda.memory_allocated(i) / 1024**2
                        reserved = torch.cuda.memory_reserved(i) / 1024**2
                        total = torch.cuda.get_device_properties(i).total_memory / 1024**2
                        print(f"[{datetime.now().strftime('%H:%M:%S')}] GPU {i}: "
                              f"할당됨: {allocated:.1f}MB / "
                              f"예약됨: {reserved:.1f}MB / "
                              f"전체: {total:.1f}MB")
            
            time.sleep(5)  # 5초마다 업데이트
            
        except Exception as e:
            print(f"모니터링 오류: {e}")
            time.sleep(5)

# Define the model name
model_name = "Salesforce/xLAM-1b-fc-r"

print("모델과 토크나이저를 로딩 중...")
# Load the model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto"
)

print("LoRA 설정 중...")
# Configure LoRA
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
)

# Get the PEFT model
model = get_peft_model(model, lora_config)

# Print the trainable parameters
print("훈련 가능한 파라미터:")
model.print_trainable_parameters()

# --- Data Preparation ---
print("데이터셋 로딩 중...")
dataset = load_dataset('csv', data_files='./data/train_data_100.csv')

# 데이터셋 구조 확인
print(f"데이터셋 구조: {dataset}")
print(f"훈련 데이터 샘플: {dataset['train'][0] if len(dataset['train']) > 0 else '빈 데이터셋'}")

# 토크나이저 함수 수정
def tokenize_function(examples):
    # query와 answers를 조합하여 입력 텍스트 생성
    input_texts = []
    
    for i in range(len(examples['query'])):
        query = examples['query'][i]
        answers = examples['answers'][i]
        tools = examples['tools'][i]
        
        # 프롬프트 형식으로 조합
        prompt = f"Query: {query}\nTools: {tools}\nAnswer: {answers}"
        input_texts.append(prompt)
    
    # 토크나이징
    tokenized_inputs = tokenizer(
        input_texts,
        padding="max_length",
        truncation=True,
        max_length=512,
        return_tensors=None
    )
    
    # Causal LM을 위한 labels 설정
    tokenized_inputs["labels"] = tokenized_inputs["input_ids"].copy()
    return tokenized_inputs

print("데이터 토크나이징 중...")
tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=dataset['train'].column_names)

# Set format for PyTorch
tokenized_datasets.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

# --- Training Arguments ---
print("훈련 설정 중...")
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=4,  # 메모리 문제 방지를 위해 줄임
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    eval_strategy="no",  # evaluation_strategy -> eval_strategy로 수정
    save_strategy="epoch",
    load_best_model_at_end=False,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=2,  # 디스크 공간 절약
    gradient_accumulation_steps=2,  # 효과적인 배치 크기 증가
    report_to=None,  # wandb 등 외부 로깅 비활성화
)

# --- Trainer ---
print("트레이너 설정 중...")
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
)


# 메모리 모니터링 시작
print("메모리 모니터링을 시작합니다...")
stop_monitoring = threading.Event()
monitor_thread = threading.Thread(target=monitor_memory, args=(stop_monitoring,))
monitor_thread.daemon = True
monitor_thread.start()

try:
    # --- Start Training ---
    print("\n" + "="*50)
    print("학습을 시작합니다...")
    print("="*50)
    
    start_time = time.time()
    trainer.train()
    end_time = time.time()
    
    print("\n" + "="*50)
    print(f"학습이 완료되었습니다!")
    print(f"총 소요 시간: {(end_time - start_time) / 60:.1f}분")
    print("="*50)
    
except Exception as e:
    print(f"\n학습 중 오류 발생: {e}")
    import traceback
    traceback.print_exc()
    
finally:
    # 모니터링 중지
    stop_monitoring.set()
    monitor_thread.join(timeout=1)
    print("\n메모리 모니터링을 중지했습니다.")

# 최종 메모리 상태 출력
print("\n최종 메모리 상태:")
memory = psutil.virtual_memory()
print(f"시스템 메모리: {memory.used / 1024**3:.1f}GB / {memory.total / 1024**3:.1f}GB ({memory.percent:.1f}%)")

if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        allocated = torch.cuda.memory_allocated(i) / 1024**2
        reserved = torch.cuda.memory_reserved(i) / 1024**2
        total = torch.cuda.get_device_properties(i).total_memory / 1024**2
        print(f"GPU {i}: {allocated:.1f}MB / {reserved:.1f}MB / {total:.1f}MB")

모델과 토크나이저를 로딩 중...
LoRA 설정 중...
훈련 가능한 파라미터:
trainable params: 1,572,864 || all params: 1,348,044,800 || trainable%: 0.1167
데이터셋 로딩 중...
데이터셋 구조: DatasetDict({
    train: Dataset({
        features: ['id', 'query', 'answers', 'tools'],
        num_rows: 100
    })
})
훈련 데이터 샘플: {'id': 0, 'query': '현재 날짜와 시간: 2025-07-28 04:40, 하우스 상태는 온도 5.1도, 습도 77.5%, 일사량 0.06, 전등 상태: 꺼짐, 최근 관수: 13시간 전, 50L, 좌측 창문: 50%, 우측 창문: 0% 이고, 적정 환경은 온도 24.4도, 습도 71.1%, 일사량 1.26 입니다.', 'answers': '[{"name": "open_left_window", "arguments": {"persent": 0}}, {"name": "light_on", "arguments": {}}]', 'tools': '[{"name": "open_left_window", "description": "왼쪽 창문을 주어진 퍼센트(%)만큼 엽니다. 온도를 낮추거나 환기시킬 때 사용합니다.", "parameters": {"persent": {"type": "int", "description": "열고자 하는 창문의 개방 정도(0-100%)"}}}, {"name": "open_right_window", "description": "오른쪽 창문을 주어진 퍼센트(%)만큼 엽니다. 온도를 낮추거나 환기시킬 때 사용합니다.", "parameters": {"persent": {"type": "int", "description": "열고자 하는 창문의 개방 정도(0-100%)"}}}, {"name": "supply_water", "description": "식물

Truncating train dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

메모리 모니터링을 시작합니다...

[11:14:00] 시스템 메모리: 사용: 14.4GB / 전체: 32.0GB (72.6%)

학습을 시작합니다...


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[11:14:05] 시스템 메모리: 사용: 21.1GB / 전체: 32.0GB (94.2%)


...


[11:18:41] 시스템 메모리: 사용: 23.1GB / 전체: 32.0GB (91.7%)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[11:18:46] 시스템 메모리: 사용: 22.8GB / 전체: 32.0GB (90.8%)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



학습이 완료되었습니다!
총 소요 시간: 4.8분

메모리 모니터링을 중지했습니다.

최종 메모리 상태:
시스템 메모리: 21.8GB / 32.0GB (87.4%)


In [10]:
trainer.model.save_pretrained("./fine_tuned_xlm_lora")
tokenizer.save_pretrained("./fine_tuned_xlm_lora")


('./fine_tuned_xlm_lora/tokenizer_config.json',
 './fine_tuned_xlm_lora/special_tokens_map.json',
 './fine_tuned_xlm_lora/chat_template.jinja',
 './fine_tuned_xlm_lora/tokenizer.json')

In [ ]:

# # Import necessary libraries
# import torch
# from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
# from peft import LoraConfig, get_peft_model
# from datasets import load_dataset
# from trl import SFTTrainer # SFTTrainer is often used for supervised fine-tuning

# # Define the model name
# model_name = "Salesforce/xLAM-1b-fc-r"

# # Load the model and tokenizer
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     # Since this is a smaller model, we might not need 4-bit quantization,
#     # but you can uncomment the following lines if you still face memory issues.
#     # load_in_4bit=True, # Load in 4-bit
#     # torch_dtype=torch.bfloat16, # Use bfloat16
#     device_map="auto" # Automatically map the model to available devices
# )

# # Configure LoRA
# lora_config = LoraConfig(
#     r=8,  # LoRA attention dimension
#     lora_alpha=16,  # Alpha parameter for LoRA scaling
#     lora_dropout=0.1,  # Dropout probability for LoRA layers
#     bias="none",  # Bias type for LoRA layers
#     task_type="CAUSAL_LM", # Task type for causal language modeling
# )

# # Get the PEFT model
# model = get_peft_model(model, lora_config)

# # Print the trainable parameters
# model.print_trainable_parameters()

# # --- Data Preparation ---
# # Load your training data from the CSV file
# dataset = load_dataset('csv', data_files='./data/train_data_100.csv')

# # You need to adapt this part based on your CSV file structure.
# # Assuming your CSV has a column named 'text' with the input sentences and a column named 'labels' with the corresponding labels.
# # You might need to add padding or truncation depending on your data and model.
# # For causal language modeling, the label is typically the next token, so the input text is also the target.
# def tokenize_function(examples):
#     # Tokenize the input text
#     tokenized_inputs = tokenizer(
#         examples['text'],
#         padding="max_length", # or padding=True, truncation=True, max_length=...
#         truncation=True,
#         return_tensors="pt"
#     )
#     # For causal language modeling, the labels are the same as the input_ids
#     tokenized_inputs["labels"] = tokenized_inputs["input_ids"].clone()
#     return tokenized_inputs

# tokenized_datasets = dataset.map(tokenize_function, batched=True)

# # If you have a validation split, you would do the same for it:
# # eval_dataset = load_dataset('csv', data_files='your_validation_data.csv')
# # tokenized_eval_datasets = eval_dataset.map(tokenize_function, batched=True)

# # Set format for PyTorch
# tokenized_datasets.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
# # if 'eval' in tokenized_eval_datasets:
# #     tokenized_eval_datasets['eval'].set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

# # --- Training Arguments ---
# training_args = TrainingArguments(
#     output_dir="./results",  # Output directory for checkpoints and logs
#     learning_rate=2e-5,
#     per_device_train_batch_size=8, # Increased batch size for smaller model
#     per_device_eval_batch_size=8,  # Increased batch size for smaller model
#     num_train_epochs=3,
#     weight_decay=0.01,
#     evaluation_strategy="no", # Set to "epoch" if you have an evaluation set
#     save_strategy="epoch",
#     load_best_model_at_end=False, # Set to True if evaluation_strategy is "epoch"
#     logging_dir="./logs",
#     logging_steps=10,
# )

# # --- Trainer ---
# # Using SFTTrainer for supervised fine-tuning is often more convenient
# trainer = SFTTrainer(
#     model=model,
#     args=training_args,
#     train_dataset=tokenized_datasets['train'], # Your training dataset
#     # eval_dataset=tokenized_eval_datasets['eval'],   # Your evaluation dataset (optional)
#     tokenizer=tokenizer, # Pass the tokenizer to the trainer
#     # data_collator=data_collator, # Optional data collator
#     # max_seq_length=..., # Optional: Set a max sequence length
# )


# # --- Start Training ---
# trainer.train()




# print("Training code updated for Salesforce/xLAM-1b-fc-r. Please run the cell to start training.")